# Deeplearning Tutorial with PyTorch

## Tensors

### Basic Operation of Tensors

In [21]:
import torch

In [22]:
x = torch.Tensor(2, 3)

In [23]:
x = torch.rand(2, 3)
y = torch.rand(2, 3)
x + y


 1.4220  0.7043  1.5416
 1.2913  0.9845  0.4548
[torch.FloatTensor of size 2x3]

In [24]:
x[:, 1]


 0.5523
 0.4161
[torch.FloatTensor of size 2]

In [25]:
x = torch.zeros(2, 3)
y = torch.ones(2, 3)
print(x, y)


 0  0  0
 0  0  0
[torch.FloatTensor of size 2x3]
 
 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]



### Numpy Bridge

In [26]:
a = torch.ones(2, 3).numpy()
print(a)

[[ 1.  1.  1.]
 [ 1.  1.  1.]]


In [27]:
import numpy as np
b = torch.from_numpy(np.ones((2, 3)))
print(b)        


 1  1  1
 1  1  1
[torch.DoubleTensor of size 2x3]



In [28]:
a = np.ones((2, 3))
b = torch.from_numpy(a)
print(a, b)
a[1, 1] = 2
print(a, b)

[[ 1.  1.  1.]
 [ 1.  1.  1.]] 
 1  1  1
 1  1  1
[torch.DoubleTensor of size 2x3]

[[ 1.  1.  1.]
 [ 1.  2.  1.]] 
 1  1  1
 1  2  1
[torch.DoubleTensor of size 2x3]



### Cuda Tensor

In [33]:
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    print(x + y) 


 1  1  1
 1  1  1
[torch.cuda.FloatTensor of size 2x3 (GPU 0)]



### Autograd

In [34]:
from torch.autograd import Variable

In [37]:
x = Variable(torch.ones(2, 2), requires_grad = True)
y = x + 2
z = y * y * 3
out = z.mean()
print("x:",x)
print("y:",y)
print("z:",z)
print("out:",out)
out.backward()
x.grad

x: Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

y: Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

z: Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]

out: Variable containing:
 27
[torch.FloatTensor of size 1]



Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

In [47]:
x = Variable(torch.randn(3), requires_grad = True)
y = x * 2
i = 1
while y.data.norm() < 1000:
    y = y * 2
    i += 1
print("y:",y)
print("i:",i)
gradient = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradient)
print("x.grad:",x.grad)

y: Variable containing:
 1327.5786
 -834.2202
 -191.4208
[torch.FloatTensor of size 3]

i: 10
x.grad: Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]



### Neural Networks

In [78]:
import torch.nn as nn
import torch.nn.functional as F

In [80]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1: ]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
net
    

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)

In [83]:
params = list(net.parameters())
print(len(params),params[0].size())

10 torch.Size([6, 1, 5, 5])


In [94]:
inputs = Variable(torch.randn(1, 1, 32, 32), requires_grad = True)
out = net(inputs)
out = out / out.sum().data[0]
print(out, out.sum())

Variable containing:
-0.1060  0.1505  0.3014  0.1564 -0.1013 -0.0137  0.0363  0.0997  0.1849  0.2918
[torch.FloatTensor of size 1x10]
 Variable containing:
 1
[torch.FloatTensor of size 1]



In [95]:
net.zero_grad()
out.backward(torch.randn(1, 10))
print(inputs.grad)

Variable containing:
(0 ,0 ,.,.) = 
  2.1711e-04 -1.0686e-03 -9.4570e-04  ...  -1.2074e-04  5.9767e-05 -4.1686e-05
 -1.3669e-04 -4.2723e-04 -2.6731e-04  ...  -2.2974e-04 -6.8316e-05 -6.5062e-05
 -1.5877e-04 -1.3391e-03 -1.7077e-03  ...   4.0821e-04 -2.3400e-04  1.6428e-04
                 ...                   ⋱                   ...                
 -9.0309e-05 -2.9015e-04 -9.9714e-04  ...   4.5496e-04 -3.8864e-04 -8.4755e-05
  5.0910e-05  1.3562e-04 -1.4279e-04  ...  -5.0996e-04  4.8594e-04 -1.8017e-04
  1.4996e-04  2.2590e-04  2.3333e-04  ...   1.2016e-04  6.2832e-05  1.5333e-05
[torch.FloatTensor of size 1x1x32x32]

